# DQN

Deep Q-Network (DQN) [[4](#Materia%C5%82y)] jest to adaptacja algorytmu Q-learningu dla głębokich sieci neuronowych. 

Celem algorytmu jest wykorzystanie głębokiej sieci neuronowej do aproksymacji optymalnej funkcji akcji-wartości (ang. *action-value function*)

$$Q^*(s,a) = \max_{\pi} \mathbb{E}(G_t\ |\ s_t=s, a_t=a, \pi) = \max_{\pi} \mathbb{E}(r_t + \gamma r_{t+1} + \gamma^2r_{t+2} + \cdots\ |\ s_t=s, a_t=a, \pi),$$

tzn. maksimum sum nagród $r_t$ uwzględniających zniżki $\gamma$ (ang. *discount factor*) w kolejnych krokach czasu $t$, które są wynikiem wybranych akcji $a$ przez taktykę (ang. *policy*) $\pi = P(a|s)$ na podstawie obserwacji $s$.

Idea jaka stoi za DQN to:
1. Wykorzystanie zrandomizowanych powtórek doświadczenia (ang *experience replay*), dzięki czemu usuwa się korelację dla kolejnych obserwacji.
2. Iterowane aktualizacje funkcji akcji-wartości $Q$ w kierunku wartości celu $y$ (ang. *target value*), wyznaczonych przez aktualizowaną okresowo (co pewną liczbę kroków) kopię sieci neuronowej z DQN.

Algorytm DQN [[4](#Materia%C5%82y)] wygląda następująco:
![DQN Algorithm](./images/dqn_algorithm.png)

# Double DQN

Double DQN [[5](#Materia%C5%82y)] jest modyfikacją algorytmu DQN, dzięki której można zapobiec zbyt optymistycznej estymacji funkcji akcji-wartości $Q$.

Modyfikacja ta polega na zmianie liczenia wartości celu $y$, tzn.
zamiast 

$$y_j = r_j + \gamma \max_{a'} \hat{Q}(\phi_{j+1}, a'; \theta^{-})$$

stosuje się

$$y_j = r_j + \gamma \hat{Q}(\phi_{j+1}, \textit{argmax}_{a'} Q(\phi_{j+1},a';\theta); \theta^{-}).$$

Nadal wykorzystuje się tutaj taktykę zachłanną (ang. *greedy policy*) przy wyborze akcji, ale zmienia się zestaw parametrów, które stosuje się dla wyboru akcji.

# Dueling DQN

Dueling DQN [[5](#Materia%C5%82y)] jest kolejną modyfikacją algorytmu DQN. W tym rozwiązaniu chodzi o nauczenie się, który stan (nie) jest wartościowy bez nauki o efekcie każdej akcji dla każdego stanu (nie zawsze jest sens wyznaczać wartość każdej z akcji, bo część z nich może nie mieć negatywnego wpływu na to co się później wydarzy). W tym celu wykorzystuje się funkcję korzyści (ang. *advantage function*)

$$A(s,a) = Q(s,a) - V(s),$$

która opisuje istotność wyboru każdej z akcji dla danego stanu $s$.

Aby uwzględnić funkcję korzyści w algorytmie, modyfikuje się sieć neuronową zastępując ostatnią warstwę dwoma strumieniami, które odpowiednio wyznaczają funkcję korzyści $A$ oraz wartości $V$, a następnie łączą się w pojedynczy output dla funkcji akcji-wartości $Q$. 

Sposoby łączenia strumieni to:
1. $Q(s,a;\theta,\alpha,\beta) = V(s;\theta,\beta) + A(s,a;\theta,\alpha)$
2. $Q(s,a;\theta,\alpha,\beta) = V(s;\theta,\beta) + (A(s,a;\theta,\alpha) - \max_{a'\in \mathcal{A}} A(s,a';\theta,\alpha))$
3. $Q(s,a;\theta,\alpha,\beta) = V(s;\theta,\beta) + (A(s,a;\theta,\alpha) - \frac{1}{|\mathcal{A}|}\sum_{a'\in \mathcal{A}} A(s,a';\theta,\alpha))$

Porównanie struktury modelu DQN z Dueling DQN na przykładzie konwolucyjnych sieci neuronowych [[5](#Materia%C5%82y)]:
![Dueling DQN](./images/dueling_dqn.png)

### Uwaga

Powyższe modyfikacje można stosować jednocześnie. 

![DQN Meme](./images/dqn_meme.jpg)

# Biblioteki i funkcje pomocnicze

In [ ]:
from PIL import Image
import numpy as np
import pandas as pd
import gym
import json
import time
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, Permute, Input
from keras.optimizers import Adam
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy, BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
    
def tournament(agent, env, action_repetition=1, nb_episodes=20, nb_max_episode_steps=500, visualize=True):
    np.random.seed(nb_episodes)
    env.seed(nb_episodes)
    tournament_results = agent.test(env, nb_episodes=nb_episodes, action_repetition=action_repetition,
                                    nb_max_episode_steps=nb_max_episode_steps, visualize=visualize)
    mean_reward = np.mean(tournament_results.history['episode_reward'])
    # env.close()
    print('Mean reward {}'.format(mean_reward))

def logs_visualizer(log_path, train=True, stats=None, fig_size=None, save_to_file=False):
    with open(log_path, 'r') as f:
        data = json.load(f)
    data = pd.DataFrame(data)
    if train:
        which_rows = data.isna().apply(lambda row: not pd.np.any(row), axis=1).tolist()
        data = data.loc[which_rows]
    episodes = data['episode']
    if stats is None:
        stats = ['duration', 'nb_episode_steps', 'episode_reward']
    if train:
        stats += ['mean_q', 'mean_eps']

    n = len(stats)
    if fig_size is None:
        fig_size = (15., 5. * n)

    _, plots = plt.subplots(nrows=n, sharex=True, figsize=fig_size)
    for i, stat in enumerate(stats):
        plots[i].plot(episodes, data[stat])
        plots[i].set_ylabel(stat, fontsize=20)
        plots[i].tick_params(labelsize=15)
    plt.xlabel('episodes', fontsize=20)
    plt.tight_layout()
    if save_to_file:
        plt.savefig(log_path[:-4] + 'pdf')
    else:
        plt.show()

# Konfiguracja gry

CartPole | MountainCar
:----:|:----:
![CartPole](./images/cartpole.png)  |  ![MountainCar](./images/mountaincar.png)


#### OpenAI Gym Wiki
- [CartPole](https://github.com/openai/gym/wiki/CartPole-v0)
- [MountainCar](https://github.com/openai/gym/wiki/MountainCar-v0)

In [ ]:
env_name = 'CartPole-v0'
# env_name = 'MountainCar-v0'
env = gym.make(env_name)

training_seed = 20
env.seed(training_seed)
np.random.seed(training_seed)
nb_actions = env.action_space.n
space_shape = env.observation_space.shape

# Parametry do modyfikacji

#### Model

`window_length` - ilość kolejnych obrazków branych jako input; w grach 2 klatki odpowiadają za ruch, 3 za prędkość

#### Memory Replay

`memory_limit` - maksymalna liczba zapamiętanych ostatnich klatek gry

#### Policy

Do wyboru z policy mamy:  
`BoltzmannQPolicy` - Boltzmann Policy opiera się na prawdopodobieństwach wyznaczonych na podstawie Q wartości. Zwraca wylosowaną akcję na podstawie obliczonych prawdopodobieństw.  

`EpsGreedyQPolicy` - $\varepsilon$-Greedy Policy wybiera akcję według schematu:  
1. Losowana jest liczba $x\sim U([0,1])$
2. Wybierana jest akcja $$a = \cases{\text{losowa akcja} & \text{ dla } x < \varepsilon\\ \textit{argmax}_{a'\in A}Q(s,a') & \text{ dla } x \ge \varepsilon}$$  

`LinearAnnealedPolicy` - Linear Annealed Policy wykorzystuje $\varepsilon$-Greedy Policy, gdzie dla danego kroku $t$ $\varepsilon_t$ wyznaczony jest według wzoru:
$$\varepsilon_t = \max{\left(\varepsilon_{min}, -\frac{\varepsilon_{max} - \varepsilon_{min}}{\text{nb_steps_annealing}}\cdot \text{nb_step}_t + \varepsilon_{max}\right)}$$  

Parametry w policy:

`eps_training_max` - początkowa wartość $\varepsilon$  
`eps_training_min` - końcowa wartość $\varepsilon$  
`eps_test` - wartość $\varepsilon$ dla testowania agenta  
`nb_steps_annealing` - liczba kroków potrzebnych do otrzymania `eps_training_min`

#### Agent

`double_dqn` - czy ma być wykorzystany Double DQN  
`dueling_dqn` - czy ma być wykorzystany Dueling DQN  
`dueling_type` - rodzaj agregacji w Dueling DQN  
`nb_steps_learning` - liczba kroków nauki agenta  
`nb_steps_warmup` - liczba kroków rozgrzewkowych, które nie wpływają na aktualizację modelów w agencie  
`nb_steps_target_model_update` - co ile kroków ma być zaktualizowany model do wyznaczania targetów  
`batch_size` - rozmiar batchy  
`gamma` - `discount factor` $\gamma$ z Q Learning  
`train_interval` - co ile kroków model ma się uczyć   
`action_repetition` - ile razy ma być wykonana pojedyncza akcja  
`optimizer_learning_rate` - learning rate w optimizerze (Adam) sieci neuronowej (śmiało można zmienić Adama na coś innego)  
`optimizer_metrics` - lista zawierająca miary w optimizerze (Adam) sieci neuronowej (śmiało można zmienić Adama na coś innego)   
`nb_steps_log` - co ile kroków mają być logowane postępy oraz zapisywane wagi modelów  
`visualize_learning` - czy gra ma być wyświetlana podczas nauki  

In [ ]:
# Model
window_length = 1

# Memory Replay
memory_limit = 50000

# Policy
eps_training_max = 1
eps_training_min = 0.1
eps_test = 0.05
nb_steps_annealing = 25000

# Agent
double_dqn = False
dueling_dqn = False
dueling_type = 'avg'
nb_steps_learning = 50000
nb_steps_warmup = 5000
nb_steps_target_model_update = 1000
batch_size=32
gamma=.99
train_interval=1
action_repetition = 1
optimizer_learning_rate = 0.01
optimizer_metrics = ['mae']
nb_steps_log = 10000
visualize_learning = False

#### Przykładowy model - można go dowolnie modyfikować

In [ ]:
input_shape = (window_length,) + space_shape
model = Sequential() 
model.add(Flatten(input_shape=input_shape))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

print('Input shape: {}'.format(input_shape))
print('Output shape: {}'.format(nb_actions))
# (model.summary())

# Nauka agenta

In [ ]:
# Memory Replay
memory = SequentialMemory(limit=memory_limit, 
                          window_length=window_length)

# Policy
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), 
                              attr='eps', 
                              value_max=eps_training_max, 
                              value_min=eps_training_min, 
                              value_test=eps_test,
                              nb_steps=nb_steps_annealing)

# Agent
dqn = DQNAgent(model=model, 
               memory=memory, 
               policy=policy,
               enable_double_dqn=double_dqn, 
               enable_dueling_network=dueling_dqn, 
               dueling_type=dueling_type,
               nb_actions=nb_actions, 
               nb_steps_warmup=nb_steps_warmup,
               target_model_update=nb_steps_target_model_update,
               batch_size=batch_size,
               gamma=gamma,
               train_interval=train_interval)
dqn.compile(Adam(lr=optimizer_learning_rate), 
            metrics=['mae'])

# Logs & weights
weights_filename = 'weights/{}_weights.h5f'.format(env_name)
log_filename = 'logs/{}_log.json'.format(env_name)
checkpoint_weights_filename = 'weights/' + env_name + '_weights_{step}.h5f'
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=nb_steps_log)]
callbacks += [FileLogger(log_filename, interval=nb_steps_log)]

# Learning
print('Start learning: ' + time.asctime())
dqn.fit(env, 
        callbacks=callbacks, 
        nb_steps=nb_steps_learning, 
        log_interval=nb_steps_log, 
        action_repetition=action_repetition, 
        visualize=visualize_learning)
print('End learning: ' + time.asctime())
logs_visualizer('./logs/CartPole-v0_log.json')

# Turniej Cartpole

## Zasady

Turniej składa się z 20 gier. Zwycięzcą zostaje osoba, która uzyska średnio najwyższy wynik.

Nagrody:
* +1 za przeżycie w danym kroku

Uwaga:
Ziarna generatorów liczb pseudolosowych są ustawione na liczbę gier (20).

### Parametry do modyfikacji

`action_repetition` - ile razy ma być wykonana pojedyncza akcja  
`visualize` - czy gra ma być wizualizowana

### Uwaga

Istnieje możliwość wczytania wcześniejszych wersji nauczonych wag modelu. Można to zrobić metodą `agent.load_weights(filepath)`, która jako argument przyjmuje ścieżkę do pliku z wagami modelu, np.:
```python
dqn.load_weights('./weights/last_weights.h5f')
```

In [ ]:
tournament_action_repetition = 1
tournament_visualize = True

In [ ]:
tournament(dqn, env, tournament_action_repetition, visualize=tournament_visualize)

# Materiały

1. [UCL Course on RL - David Silver, 2015](http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching.html)
2. [Deep RL Bootcamp, sierpień 2017](https://sites.google.com/view/deep-rl-bootcamp/)
3. [Reinforcement Learning: An Introduction (draft książki, maj 2018), Richard S. Sutton and Andrew G. Barto](https://drive.google.com/file/d/1xeUDVGWGUUv1-ccUMAZHJLej2C7aAFWY/view)
4. [(DeepMind DQN) Human-level control through deep reinforcement learning](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf)
5. [(DeepMind Double DQN) Deep Reinforcement Learning with Double Q-learning](https://arxiv.org/pdf/1509.06461.pdf)
6. [(DeepMind Dueling DQN) Dueling Network Architectures for Deep Reinforcement Learning](https://arxiv.org/pdf/1511.06581.pdf)
7. [OpenAI Gym](https://gym.openai.com)
8. [OpenAI Gym Github](https://github.com/openai/gym)
9. [OpenAI Baselines Github](https://github.com/openai/baselines)
10. [keras-rl Github](https://github.com/keras-rl/keras-rl)
11. [TensorForce Github](https://github.com/reinforceio/tensorforce)